In [1]:
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm.auto import tqdm

In [5]:
file = 'пробник_авокадо.xlsx'

mpstats_df = pd.read_excel(file)

In [6]:
articul_url =mpstats_df.URL

In [7]:
# важно! данные парсятся из Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run the browser in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

In [8]:
def parse_articul(url, driver):
    driver.get(url)
    try:
        # Wait for the page to load
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.catalog_main_table')))

        # Expand all collapsible and expandable elements
        collapsible_buttons = driver.find_elements_by_css_selector('.collapsable__header')
        for button in collapsible_buttons:
            button.click()
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.collapsable__content.j-description')))
        page_source = driver.page_source
    except:
        page_source = driver.page_source
        
    soup = BeautifulSoup(page_source, 'html.parser')

    articul_data = {}
    
    articul_data['url'] = url
    articul_data['Материал']=soup.find('span', {'data-link':'text{:selectedNomenclature^consist}'}).text

    attribute_elements = soup.find_all('tr', {'class':"product-params__row"})
    


    for elem in attribute_elements:
        subelements = elem.find_all('span')
        key = subelements[1].text
        value = subelements[2].text
        articul_data[key]=value
        
    return articul_data

In [10]:
urllist = list(articul_url)

In [66]:
df_discription = pd.DataFrame(results)
df_discription = df_discription.rename(columns={'url':'URL'})

In [68]:
# объединение данных старой таблички и новой

all_data = pd.merge(pants_mpstats_df, df_discription, on='URL')
all_data.head()

,Search position,SKU,Name,Category,Brand,Seller,Color,Balance,Balance FBS,Comments,...,Материал игрушки,Назначение подарка,Повод,Страна производства,Ширина предмета,Эффекты,Комплектация,Типоразмер элемента питания,Назначение,Вес товара с упаковкой (г)
0,15,40219927,"Игрушка мягкая Стич и Ангел / детская, плюшева...",Игрушки/Мягкие игрушки,BimbaMania,ИП Костянов Данила Алексеевич,синий,231,0,469,...,ткань подушка сна патруль динозавр умный едино...,для ребенка; подруге; любимой,Подарок на Новый год / День рождения / Рождест...,Китай,25 см,фигурка новая лалафанфан жирафик батон безопас...,"Мягкая игрушка ""Стич"" - 1 шт",сиреноголовый хагги вагги huggy wuggy самолет ...,NaN,NaN
1,7,40219928,"Игрушка мягкая Стич и Ангел / детская, плюшева...",Женщинам/Подарки женщинам/Игрушки/Мягкая игрушка,BimbaMania,ИП Костянов Данила Алексеевич,розовый,70,0,469,...,ткань подушка сна патруль динозавр умный едино...,для ребенка; подруге; любимой,Подарок на Новый год / День рождения / Рождест...,Китай,25 см,фигурка новая лалафанфан жирафик батон безопас...,"Мягкая игрушка ""Стич"" - 1 шт",сиреноголовый хагги вагги huggy wuggy самолет ...,NaN,NaN
2,10,11717775,Мягкая игрушка подушка Авокэт 39 см,Женщинам/Подарки женщинам/Игрушки/Мягкая игрушка,Fancy,ООО Аспект 6703004480,"зеленый,белый,коричневый,черный",160,0,381,...,текстиль,для ребенка; для подруги; любимой девушке маме...,день рождения; 8 марта; 14 Февраля,Беларусь,30 см,игрушка для сна; плюшевая; игрушка в подарок,мягкая игрушка,NaN,Игрушка подушка антистресс; Пушистая игрушка д...,491 г
3,33,32990071,Авокадо мягкая игрушка 40см,Женщинам/Подарки женщинам/Игрушки/Мягкая игрушка,GoldStitch,ООО Эксклюзивери РУС,зеленый,131,2,71,...,Плюшевая ткань гипоаллергенная,для ребенка; девушке; подруге,8 марта; день рождения; Выпускной,Россия,30 см,Антистресс,Мягкая игрушка - 1 шт,NaN,для ребенка; для сна; Подарок,500 г
4,38,90220991,"Большая мягкая игрушка с пледом, подушка, плюш...",Мужчинам/Подарки мужчинам/Игрушки,ХОХОЧУ,ИП Назаров Сергей Александрович,зеленый,0,74,578,...,текстиль; плюш; плюшевая игрушка,для ребенка; подруге; любимому,23 февраля; день рождения; день Святого Валентина,Россия,65 см,Антистресс; Подушка обнимашка; Мягкая игрушка ...,Мягкая игрушка подушка 3 в 1 с пледом внутри -...,Без питания; С одеялом; Игрушка для сна,символ года; 8 Марта; игрушка сплюшка,1000 г


In [54]:
# можно выбрать только определенные колонки без лишних и сохранить итоговую таблицу

final_data = all_data[['SKU','Name','Brand', 'Seller','Color','Comments', 'Final price','Sales', 'Revenue','Lost profit',
                       'URL', 'Thumb','Country','First Date','Вид мягкой игрушки','Наполнитель','Материал игрушки','Эффекты']]

In [55]:
final_data.head()

,SKU,Name,Brand,Seller,Color,Comments,Final price,Sales,Revenue,Lost profit,URL,Thumb,Country,First Date,Вид мягкой игрушки,Наполнитель,Материал игрушки,Эффекты
0,40219927,"Игрушка мягкая Стич и Ангел / детская, плюшева...",BimbaMania,ИП Костянов Данила Алексеевич,синий,469,1011,1014,1014781,22804.0674,https://www.wildberries.ru/catalog/40219927/de...,https://basket-03.wb.ru/vol402/part40219/40219...,Китай,2021-09-13,Стич и Лило; фигурки животных; персонаж мультф...,Плюш синтепон; плотная набивка с шариками; син...,ткань подушка сна патруль динозавр умный едино...,фигурка новая лалафанфан жирафик батон безопас...
1,40219928,"Игрушка мягкая Стич и Ангел / детская, плюшева...",BimbaMania,ИП Костянов Данила Алексеевич,розовый,469,1011,555,555506,18937.7045,https://www.wildberries.ru/catalog/40219928/de...,https://basket-03.wb.ru/vol402/part40219/40219...,Китай,2021-09-13,Стич и Лило; фигурки животных; персонаж мультф...,Плюш синтепон; плотная набивка с шариками; син...,ткань подушка сна патруль динозавр умный едино...,фигурка новая лалафанфан жирафик батон безопас...
2,11717775,Мягкая игрушка подушка Авокэт 39 см,Fancy,ООО Аспект 6703004480,"зеленый,белый,коричневый,черный",381,1011,305,305655,0.0000,https://www.wildberries.ru/catalog/11717775/de...,https://basket-01.wb.ru/vol117/part11717/11717...,Беларусь,2020-03-30,Кот; Игрушка подушка; авокадо мягкая игрушка,полиэфирное волокно,текстиль,игрушка для сна; плюшевая; игрушка в подарок
3,32990071,Авокадо мягкая игрушка 40см,GoldStitch,ООО Эксклюзивери РУС,зеленый,71,1596,79,122993,0.0000,https://www.wildberries.ru/catalog/32990071/de...,https://basket-03.wb.ru/vol329/part32990/32990...,Россия,2021-06-29,Авокадо; Игрушка обнимашка; Игрушка подушка,холлофайбер,Плюшевая ткань гипоаллергенная,Антистресс
4,90220991,"Большая мягкая игрушка с пледом, подушка, плюш...",ХОХОЧУ,ИП Назаров Сергей Александрович,зеленый,578,1634,50,85736,70303.5200,https://www.wildberries.ru/catalog/90220991/de...,https://basket-05.wb.ru/vol902/part90220/90220...,Россия,2022-06-16,Плюшевая игрушка; животные плюшевые; Игрушка п...,синтепух; пух,текстиль; плюш; плюшевая игрушка,Антистресс; Подушка обнимашка; Мягкая игрушка ...


In [56]:
# final_data.to_excel('характеристики_авокадо(.xlsx')